In [1]:
import pandas as pd
import numpy as np
import pickle
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from project_utils import runModel, split_and_scale
from sklearn import feature_selection
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.base import ClassifierMixin
from numpy import ndarray
import time
from project_utils import runModel, runModelCV, training_with_PCA, scaling, runModelStepwiseSelection
from scipy.stats import uniform
from sklearn.model_selection import RepeatedStratifiedKFold, RandomizedSearchCV
np.random.seed(42)

In [2]:
#Set data for binary classification

df = pd.read_csv('npf_train.csv')
df = df.drop(columns=['date', 'id', 'partlybad'])
df['class2'] = (df['class4'] == 'nonevent').astype(int)
X = df.drop(columns=['class4', 'class2'])
#X = X[["CO2168.mean", "CO242.mean", "CO2504.mean"]]
y = df['class2']



In [3]:
#Define your model parametrs here and add them to the list

rf = RandomForestClassifier(n_estimators=50, random_state=42, class_weight={1: 0.55, 0:0.45}, min_samples_leaf=5, min_samples_split=5)
lr = LogisticRegressionCV(max_iter=10)
nb = GaussianNB()
svm = SVC(probability=True)
knn = KNeighborsClassifier(n_neighbors=5)
#xcols = xdf.columns

models = [rf, lr, nb, svm, knn]

In [4]:
results = []
for m in models:
    results.append(runModelStepwiseSelection(m, X, y))

/Users/kukkojoo/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


RandomForestClassifier(class_weight={0: 0.45, 1: 0.55}, min_samples_leaf=5,
                       min_samples_split=5, n_estimators=50, random_state=42) 588.185201883316
LogisticRegressionCV(max_iter=10) 854.709951877594
GaussianNB() 17.87069010734558
SVC(probability=True) 168.50346302986145
KNeighborsClassifier() 34.33825492858887


In [46]:
def splitAndScaleDate(X, y, split=0.33):
    scaler = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    return X_train, X_test, y_train, y_test

In [47]:
def runModel(m, X_, y_):
    time_start = time.time()
    classifier_pipeline = make_pipeline(StandardScaler(), m)
    
    cv = KFold(n_splits=10, random_state=0, shuffle=False)
    
    sfs1 = SFS(classifier_pipeline, 
        k_features=(1, 100), 
        forward=True, 
        scoring='accuracy',
        cv=cv,
        n_jobs=-1
    )
    sfs1.fit(X_,y_)
    time_end = time.time()
    time_run = time_end - time_start
    print('best combination (ACC: %.3f): %s\n' % (sfs1.k_score_, sfs1.k_feature_idx_), time_run)
    
    d = {"model": m, "score": sfs1.k_score_, "combination": sfs1.k_feature_idx_}
    return d

In [21]:
time_start = time.time()
#X_train, X_test, y_train, y_test = splitAndScaleDate(X, y)
scores = [runModel(m, X.iloc[:, 0:100], y) for m in models]
results = pd.DataFrame(scores)
time_end = time.time()
print(time_end - time_start)
results.head()

/Users/kukkojoo/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


best combination (ACC: 0.889): (3, 12, 20, 23, 29, 31, 34, 40, 43, 53, 69, 70, 77, 80, 83, 92, 96)
 626.5888030529022
best combination (ACC: 0.882): (0, 2, 4, 5, 6, 9, 10, 11, 12, 17, 18, 20, 25, 31, 32, 33, 39, 42, 47, 48, 49, 50, 52, 53, 54, 56, 57, 61, 62, 63, 64, 67, 69, 74, 75, 76, 79, 85, 89, 91, 94, 98)
 848.2930130958557
best combination (ACC: 0.865): (0, 4, 10, 11, 12, 16, 18, 48, 49, 51, 53, 54, 55, 57, 58, 69, 70, 84, 86, 98, 99)
 18.885116815567017
best combination (ACC: 0.898): (10, 12, 15, 17, 21, 24, 50, 67, 68, 70, 71, 80, 98, 99)
 172.72309827804565
best combination (ACC: 0.898): (16, 70, 71, 80, 82, 98)
 35.30685496330261
1701.8022410869598


,model,score,combination
0,"RandomForestClassifier(class_weight={0: 0.45, ...",0.888792,"(3, 12, 20, 23, 29, 31, 34, 40, 43, 53, 69, 70..."
1,LogisticRegressionCV(max_iter=10),0.882271,"(0, 2, 4, 5, 6, 9, 10, 11, 12, 17, 18, 20, 25,..."
2,GaussianNB(),0.864879,"(0, 4, 10, 11, 12, 16, 18, 48, 49, 51, 53, 54,..."
3,SVC(probability=True),0.897585,"(10, 12, 15, 17, 21, 24, 50, 67, 68, 70, 71, 8..."
4,KNeighborsClassifier(),0.897536,"(16, 70, 71, 80, 82, 98)"


### Time
* 20 -> 38s
* 30 -> 85s
* 40 -> 155s best score: 0.856
* 50 -> 244s best score: 0.853
* 60 -> 358s best score: 0.856
* 70 -> 488s best score: 0.880
* 80 -> 644s best score: 0.884
* 90 -> 834s best score: 0.893

In [22]:
# 101
#max = 0
#best_shape = 0
#best_model = ""
#for i in range(1, 101):
#    #print(df_.iloc[:, 0:i].shape)
#    #print(i)
#    X_train, X_test, y_train, y_test = splitAndScaleDate(df_.iloc[:, 0:i], y)
#    scores = [runModel(m, X_train, X_test, y_train, y_test) for m in models]
#    results = pd.DataFrame(scores)
#    test_scores = results['test score'].to_numpy()
#    
#    if (np.amax(test_scores) > max):
#        max = np.amax(test_scores)
#        best_shape = df_.iloc[:, 0:i].shape
#        
#print(max)
#print(best_shape)

In [152]:
print(df_.iloc[:, 0:101].shape)
X_train, X_test, y_train, y_test = splitAndScaleDate(df_.iloc[:, 0:101], y)
scores = [runModel(m, X_train, X_test, y_train, y_test) for m in models]
results = pd.DataFrame(scores)
results.head()

(458, 100)


,model,train score,test score
0,"(DecisionTreeClassifier(max_features='auto', m...",0.944444,0.855263
1,LogisticRegressionCV(max_iter=1000),0.918301,0.815789
2,GaussianNB(),0.810458,0.763158
3,SVC(probability=True),0.921569,0.835526
4,KNeighborsClassifier(),0.911765,0.815789


In [4]:
svm = SVC(probability=True)

In [5]:
classifier_pipeline = make_pipeline(StandardScaler(), svm)
cv = KFold(n_splits=10, random_state=0, shuffle=False)

/Users/kukkojoo/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [6]:
sfs1 = SFS(classifier_pipeline, 
           k_features=(1, 20), 
           forward=True, 
           scoring='accuracy',
           cv=cv,
           n_jobs=-1
          )

In [7]:
time_start = time.time()
sfs1.fit(X,y)
time_end = time.time()
print(time_end - time_start)

37.27081799507141


In [8]:
sfs1.subsets_

{1: {'feature_idx': (70,),
  'cv_scores': array([0.80434783, 0.84782609, 0.84782609, 0.7173913 , 0.7826087 ,
         0.84782609, 0.7826087 , 0.63043478, 0.86666667, 0.73333333]),
  'avg_score': 0.7860869565217391,
  'feature_names': ('RHIRGA42.mean',)},
 2: {'feature_idx': (12, 70),
  'cv_scores': array([0.80434783, 0.86956522, 0.84782609, 0.76086957, 0.86956522,
         0.91304348, 0.84782609, 0.86956522, 0.91111111, 0.84444444]),
  'avg_score': 0.853816425120773,
  'feature_names': ('H2O336.mean', 'RHIRGA42.mean')},
 3: {'feature_idx': (12, 67, 70),
  'cv_scores': array([0.86956522, 0.76086957, 0.86956522, 0.7826087 , 0.84782609,
         0.93478261, 0.86956522, 0.82608696, 0.97777778, 0.86666667]),
  'avg_score': 0.8605314009661835,
  'feature_names': ('H2O336.mean', 'RHIRGA168.std', 'RHIRGA42.mean')},
 4: {'feature_idx': (12, 67, 70, 80),
  'cv_scores': array([0.84782609, 0.7826087 , 0.89130435, 0.7826087 , 0.86956522,
         0.91304348, 0.91304348, 0.82608696, 0.95555556, 0.86

In [9]:
print('best combination (ACC: %.3f): %s\n' % (sfs1.k_score_, sfs1.k_feature_idx_))

best combination (ACC: 0.898): (10, 12, 15, 17, 21, 24, 50, 67, 68, 70, 71, 80, 98, 99)



In [48]:
def optimiseModelParams(model: ClassifierMixin, paramDistributions: dict, x: ndarray, y: ndarray, n_iterations: int=20, k_folds: int=10):
    kfolds = RepeatedStratifiedKFold(n_splits=k_folds, n_repeats=n_iterations)
    searcher = RandomizedSearchCV(estimator=model, param_distributions=paramDistributions, n_iter = n_iterations, n_jobs=-1, refit=True, cv=kfolds)
    searcher.fit(x,y)
    #pickle.dump(pd.DataFrame(searcher.cv_results_), open(f'{getNiceModelName(model)}-parameter_tuning.pickle', 'wb'))
    return searcher.best_params_

In [56]:
def runModelStepwiseSelection(model: ClassifierMixin, x: ndarray, y: ndarray):
    
    modelParams = []
    
    time_start = time.time()
    
    if model == None:
        raise ValueError("no model supplied")
    if not isinstance(model, ClassifierMixin):
        warn("did you pass a valid sklearn model?")
    
    classifier_pipeline = make_pipeline(StandardScaler(), model)
    
    cv = KFold(n_splits=10, random_state=0, shuffle=False)

    sfs1 = SFS(classifier_pipeline, 
        k_features=(1, 100), 
        forward=True, 
        scoring='accuracy',
        cv=cv,
        n_jobs=-1
    )

    sfs1.fit(X,y)
    time_end = time.time()
    time_run = time_end - time_start
    print(model, time_run)
    #print('best combination (ACC: %.3f): %s\n' % (sfs1.k_score_, sfs1.k_feature_idx_), time_run)
    
    
    return {"model": model, "features": sfs1.k_feature_idx_, "scores": sfs1.k_score_}
    
    
    

In [57]:
svm = SVC(probability=True)
svm_params = dict(C=uniform(loc=0, scale=4), kernel=['sigmoid', 'rbf', 'poly'])

In [58]:
results = []
for m in models:
    results.append(runModelStepwiseSelection(m, X, y))

RandomForestClassifier(class_weight={0: 0.45, 1: 0.55}, min_samples_leaf=5,
                       min_samples_split=5, n_estimators=50, random_state=42) 596.1331169605255
LogisticRegressionCV(max_iter=10) 834.5605890750885
GaussianNB() 19.370463848114014
SVC(probability=True) 168.49733591079712
KNeighborsClassifier() 34.474915981292725


In [61]:
for r in results:
    print(r)
    print("")

{'model': RandomForestClassifier(class_weight={0: 0.45, 1: 0.55}, min_samples_leaf=5,
                       min_samples_split=5, n_estimators=50, random_state=42), 'features': (3, 12, 20, 23, 29, 31, 34, 40, 43, 53, 69, 70, 77, 80, 83, 92, 96), 'scores': 0.888792270531401}

{'model': LogisticRegressionCV(max_iter=10), 'features': (0, 2, 4, 5, 6, 9, 10, 11, 12, 17, 18, 20, 25, 31, 32, 33, 39, 42, 47, 48, 49, 50, 52, 53, 54, 56, 57, 61, 62, 63, 64, 67, 69, 74, 75, 76, 79, 85, 89, 91, 94, 98), 'scores': 0.8822705314009662}

{'model': GaussianNB(), 'features': (0, 4, 10, 11, 12, 16, 18, 48, 49, 51, 53, 54, 55, 57, 58, 69, 70, 84, 86, 98, 99), 'scores': 0.8648792270531402}

{'model': SVC(probability=True), 'features': (10, 12, 15, 17, 21, 24, 50, 67, 68, 70, 71, 80, 98, 99), 'scores': 0.897584541062802}

{'model': KNeighborsClassifier(), 'features': (16, 70, 71, 80, 82, 98), 'scores': 0.897536231884058}



In [65]:
x_t = X.iloc[:, [10, 12, 15, 17, 21, 24, 50, 67, 68, 70, 71, 80, 98, 99]]

In [60]:
s = SVC(probability=True)
X_train, X_test, y_train, y_test = splitAndScaleDate(x_t, y)

In [34]:
s.fit (X_train, y_train)

SVC(probability=True)

In [35]:
s.score(X_train, y_train)

0.8921568627450981

In [36]:
s.score(X_test, y_test)

0.875

In [84]:
s = SVC(probability=True)
runModel(s, X_train, X_test, y_train, y_test)

('SVC', 0.9084967320261438, 11.486924123757571, 0.875, 8.327975103570852)

In [66]:
svm = SVC(probability=True)
svm_params = dict(C=uniform(loc=0, scale=4), kernel=['sigmoid', 'rbf', 'poly'])

In [67]:
lr = LogisticRegression()
lr_params = dict(C=uniform(loc=0, scale=4), penalty=['l2'], solver=['newton-cg', 'lbfgs'])

In [68]:
PCA_num = 15

In [91]:
x_scaled = scaling(x_t)

In [92]:
x_t = X.iloc[:, [10, 12, 15, 17, 21, 24, 50, 67, 68, 70, 71, 80, 98, 99]]
[x_PCA, pca] = training_with_PCA(x_t,PCA_num)
runModelCV(svm, svm_params, x_scaled, y, 5, 5)

runModelCV call done


(        Train Accuracy  Train Perplex  Validation Accuracy  \
 SVC-1         0.891304       8.530523             0.863388   
 SVC-2         0.956522       8.409353             0.838798   
 SVC-3         0.923913       7.957444             0.860656   
 SVC-4         0.934066       9.300850             0.863760   
 SVC-5         0.923077      12.714039             0.847411   
 SVC-6         0.934783       9.124257             0.811475   
 SVC-7         0.945652      12.855980             0.868852   
 SVC-8         0.934783       9.785516             0.836066   
 SVC-9         0.923077       7.738531             0.869210   
 SVC-10        0.934066       9.177957             0.855586   
 SVC-11        0.923913      10.259293             0.871585   
 SVC-12        0.934783       8.047806             0.890710   
 SVC-13        0.902174       5.211004             0.877049   
 SVC-14        0.945055       9.265611             0.877384   
 SVC-15        0.934066       6.824286             0.85

In [96]:
x_t = X.iloc[:, [0, 2, 4, 5, 6, 9, 10, 11, 12, 17, 18, 20, 25, 31, 32, 33, 39, 42, 47, 48, 49, 50, 52, 53, 54, 56, 57, 61, 62, 63, 64, 67, 69, 74, 75, 76, 79, 85, 89, 91, 94, 98]]
#[x_PCA, pca] = training_with_PCA(x_t,PCA_num)
x_scaled = scaling(x_t)
runModelCV(lr, lr_params, x_scaled, y, 5, 5)

runModelCV call done


(                       Train Accuracy  Train Perplex  Validation Accuracy  \
 LogisticRegression-1         0.869565      22.254417             0.819672   
 LogisticRegression-2         0.913043      17.326000             0.855191   
 LogisticRegression-3         0.880435       9.816967             0.822404   
 LogisticRegression-4         0.901099      20.860745             0.847411   
 LogisticRegression-5         0.945055      39.194169             0.828338   
 LogisticRegression-6         0.934783      58.986569             0.844262   
 LogisticRegression-7         0.913043      15.546377             0.836066   
 LogisticRegression-8         0.934783      25.644328             0.830601   
 LogisticRegression-9         0.934066      15.786413             0.828338   
 LogisticRegression-10        0.934066      20.701636             0.825613   
 LogisticRegression-11        0.902174      12.396270             0.822404   
 LogisticRegression-12        0.891304      10.641659           

In [94]:
rf = RandomForestClassifier()
rf_params = dict(min_samples_split=[2,3,4,5,6], min_samples_leaf=[1,2,3,4,5,6])

In [97]:
x_t = X.iloc[:, [3, 12, 20, 23, 29, 31, 34, 40, 43, 53, 69, 70, 77, 80, 83, 92, 96]]
x_scaled = scaling(x_t)
#[x_PCA, pca] = training_with_PCA(x_t,PCA_num)
runModelCV(rf, rf_params, x_scaled, y, 5, 5)

runModelCV call done


(                           Train Accuracy  Train Perplex  Validation Accuracy  \
 RandomForestClassifier-1         0.913043       6.460607             0.857923   
 RandomForestClassifier-2         0.913043       6.072217             0.855191   
 RandomForestClassifier-3         0.923913       5.785159             0.830601   
 RandomForestClassifier-4         0.901099       7.118300             0.866485   
 RandomForestClassifier-5         0.890110       5.171134             0.871935   
 RandomForestClassifier-6         0.923913       5.826633             0.866120   
 RandomForestClassifier-7         0.880435       5.398261             0.822404   
 RandomForestClassifier-8         0.934783       7.460442             0.852459   
 RandomForestClassifier-9         0.901099       6.027355             0.852861   
 RandomForestClassifier-10        0.923077       7.799066             0.850136   
 RandomForestClassifier-11        0.934783       7.240443             0.844262   
 RandomForestCla

In [98]:
nb = GaussianNB()
nb_params = {}

In [99]:
x_t = X.iloc[:, [0, 4, 10, 11, 12, 16, 18, 48, 49, 51, 53, 54, 55, 57, 58, 69, 70, 84, 86, 98, 99]]
x_scaled = scaling(x_t)
#[x_PCA, pca] = training_with_PCA(x_t,PCA_num)
runModelCV(nb, nb_params, x_scaled, y, 5, 5)

/Users/kukkojoo/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 1 is smaller than n_iter=20. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


runModelCV call done


(               Train Accuracy  Train Perplex  Validation Accuracy  \
 GaussianNB-1         0.815217     279.109689             0.846995   
 GaussianNB-2         0.891304    1453.820546             0.825137   
 GaussianNB-3         0.913043   11426.027507             0.808743   
 GaussianNB-4         0.868132    1617.276681             0.825613   
 GaussianNB-5         0.846154    6172.482644             0.724796   
 GaussianNB-6         0.902174    6601.765442             0.819672   
 GaussianNB-7         0.826087     995.782342             0.838798   
 GaussianNB-8         0.858696     337.613283             0.833333   
 GaussianNB-9         0.835165     187.238424             0.828338   
 GaussianNB-10        0.879121    5088.878958             0.852861   
 GaussianNB-11        0.880435    3601.723612             0.816940   
 GaussianNB-12        0.880435    2047.100393             0.797814   
 GaussianNB-13        0.793478     211.268756             0.806011   
 GaussianNB-14      

In [103]:
knn = KNeighborsClassifier(n_neighbors=5)
knn_params = {}

In [104]:
x_t = X.iloc[:, [16, 70, 71, 80, 82, 98]]
x_scaled = scaling(x_t)
#[x_PCA, pca] = training_with_PCA(x_t,PCA_num)
runModelCV(knn, knn_params, x_scaled, y, 5, 5)

/Users/kukkojoo/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 1 is smaller than n_iter=20. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


runModelCV call done


(                         Train Accuracy  Train Perplex  Validation Accuracy  \
 KNeighborsClassifier-1         0.858696   2.622265e+07             0.877049   
 KNeighborsClassifier-2         0.913043   1.036143e+09             0.866120   
 KNeighborsClassifier-3         0.923913   3.649031e+08             0.844262   
 KNeighborsClassifier-4         0.846154   1.064405e+07             0.850136   
 KNeighborsClassifier-5         0.890110   2.285999e+07             0.836512   
 KNeighborsClassifier-6         0.902174   1.972981e+07             0.868852   
 KNeighborsClassifier-7         0.891304   8.125180e+07             0.877049   
 KNeighborsClassifier-8         0.913043   6.262261e+09             0.849727   
 KNeighborsClassifier-9         0.890110   1.804012e+06             0.863760   
 KNeighborsClassifier-10        0.934066   1.836416e+09             0.841962   
 KNeighborsClassifier-11        0.869565   1.170974e+08             0.846995   
 KNeighborsClassifier-12        0.880435